Working with Baltimore City Crime Data

In [2]:
!pip install pyspark

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, TimestampType, DateType
from pyspark.sql.functions import col,array_contains,substring

     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 52.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=543fa4ee86c0a839f0cc9bc139d7e1ea4cdae2104e4ca17ad76a75eb34d67fc0
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
!ls

Part1_Crime_data.csv  sample_data


In [5]:
spark = SparkSession.builder.appName('Balt_Crime').getOrCreate()

a) Specify the schema for the crime data set

In [6]:
df2 = spark.read.option("header",True) \
     .csv("Part1_Crime_data.csv")
print("This specifies the schema of the Baltimore Crime Data")
df2.printSchema()

This specifies the schema of the Baltimore Crime Data
root
 |-- X: string (nullable = true)
 |-- Y: string (nullable = true)
 |-- RowID: string (nullable = true)
 |-- CrimeDateTime: string (nullable = true)
 |-- CrimeCode: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Inside_Outside: string (nullable = true)
 |-- Weapon: string (nullable = true)
 |-- Post: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- GeoLocation: string (nullable = true)
 |-- Premise: string (nullable = true)
 |-- VRIName: string (nullable = true)
 |-- Total_Incidents: string (nullable = true)
 |-- Shape: string (nullable = true)



b) Read the file using schema definition

In [7]:
df2.show(5)

+----------------+----------------+-----+--------------------+---------+------------------+--------------------+--------------+-------+----+---------+--------------+--------+---------+------------------+-------+-------+---------------+-----+
|               X|               Y|RowID|       CrimeDateTime|CrimeCode|          Location|         Description|Inside_Outside| Weapon|Post| District|  Neighborhood|Latitude|Longitude|       GeoLocation|Premise|VRIName|Total_Incidents|Shape|
+----------------+----------------+-----+--------------------+---------+------------------+--------------------+--------------+-------+----+---------+--------------+--------+---------+------------------+-------+-------+---------------+-----+
|1428019.10487147|589532.731060804|    1|2022/03/05 03:43:...|      3NF|     1800 FLEET ST|    ROBBERY - STREET|          null|FIREARM| 213|SOUTHEAST|   FELLS POINT| 39.2847| -76.5913|(39.2847,-76.5913)|   null|   null|              1| null|
|1428019.10487147|589532.7310608

In [8]:
schema = StructType() \
      .add("X", DoubleType(),True) \
      .add("Y", DoubleType(),True) \
      .add("RowID",IntegerType(),True) \
      .add("CrimeDateTime",StringType(),True) \
      .add("CrimeCode",StringType(),True) \
      .add("Location",StringType(),True) \
      .add("Description",StringType(),True) \
      .add("Inside_Outside",StringType(),True) \
      .add("Weapon",StringType(),True) \
      .add("Post",IntegerType(),True) \
      .add("District",StringType(),True) \
      .add("Neighborhood",StringType(),True) \
      .add("Latitude",DoubleType(),True) \
      .add("Longtitude",DoubleType(),True) \
      .add("GeoLocation",StringType(),True) \
      .add("Premise",StringType(),True) \
      .add("VRIName",StringType(),True) \
      .add("Total_Incidents",IntegerType(),True) \
      .add("Shape",StringType(),True)

c) Cache the DataFrame

In [9]:
df2.persist()

DataFrame[X: string, Y: string, RowID: string, CrimeDateTime: string, CrimeCode: string, Location: string, Description: string, Inside_Outside: string, Weapon: string, Post: string, District: string, Neighborhood: string, Latitude: string, Longitude: string, GeoLocation: string, Premise: string, VRIName: string, Total_Incidents: string, Shape: string]

d) Show the count of rows

In [10]:
print("This counts the number of rows")
df2.count()

This counts the number of rows


291382

e) Print the schema

In [21]:
df2_with_schema = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("/content/Part1_Crime_data.csv")
print("This shows the new schema applied.")
df2_with_schema.printSchema

This shows the new schema applied.


<bound method DataFrame.printSchema of DataFrame[X: double, Y: double, RowID: int, CrimeDateTime: string, CrimeCode: string, Location: string, Description: string, Inside_Outside: string, Weapon: string, Post: int, District: string, Neighborhood: string, Latitude: double, Longtitude: double, GeoLocation: string, Premise: string, VRIName: string, Total_Incidents: int, Shape: string]>

e) Display the first 5 rows

In [14]:
print("This shows the first five lines of the dataframe")
df2_with_schema.show(5)

This shows the first five lines of the dataframe
+----------------+----------------+-----+--------------------+---------+------------------+--------------------+--------------+-------+----+---------+--------------+--------+----------+------------------+-------+-------+---------------+-----+
|               X|               Y|RowID|       CrimeDateTime|CrimeCode|          Location|         Description|Inside_Outside| Weapon|Post| District|  Neighborhood|Latitude|Longtitude|       GeoLocation|Premise|VRIName|Total_Incidents|Shape|
+----------------+----------------+-----+--------------------+---------+------------------+--------------------+--------------+-------+----+---------+--------------+--------+----------+------------------+-------+-------+---------------+-----+
|1428019.10487147|589532.731060804|    1|2022/03/05 03:43:...|      3NF|     1800 FLEET ST|    ROBBERY - STREET|          null|FIREARM| 213|SOUTHEAST|   FELLS POINT| 39.2847|  -76.5913|(39.2847,-76.5913)|   null|   null|  

Answer following questions using PySpark

1. What are distinct crime codes?

In [15]:
(df2_with_schema
.select("CrimeCode")
.where(col("CrimeCode").isNotNull())
.distinct()
.show(90, False))

+---------+
|CrimeCode|
+---------+
|3P       |
|3K       |
|3BJ      |
|1A       |
|3M       |
|5F       |
|4B       |
|3B       |
|3NF      |
|7A       |
|3EF      |
|3N       |
|5D       |
|6K       |
|3LO      |
|3AF      |
|7B       |
|3GO      |
|3AJF     |
|8GV      |
|8AO      |
|7C       |
|3AK      |
|3GK      |
|6L       |
|3EO      |
|3JO      |
|3F       |
|1K       |
|8H       |
|8CV      |
|8DO      |
|4C       |
|5A       |
|6C       |
|3NK      |
|3D       |
|6H       |
|3LK      |
|3AJK     |
|3CO      |
|3L       |
|4E       |
|8BV      |
|6D       |
|2A       |
|3C       |
|8I       |
|3NO      |
|3JF      |
|3LF      |
|3E       |
|1O       |
|8J       |
|3CK      |
|8BO      |
|2B       |
|3JK      |
|5B       |
|4A       |
|8GO      |
|8EV      |
|3CF      |
|8EO      |
|6G       |
|6A       |
|9S       |
|3EK      |
|8FV      |
|3GF      |
|8CO      |
|3H       |
|4D       |
|8FO      |
|6J       |
|6F       |
|6E       |
|3J       |
|5C       |
|5G       |
|8AV

2. Count the number of crimes by the crime codes and order by the resulting counts in descending order

In [16]:
(df2_with_schema 
.select("CrimeCode")
.where(col("CrimeCode").isNotNull())
.groupBy("CrimeCode")
.count()
.orderBy("count", ascending=False)
.show(n=90, truncate=False))

+---------+-----+
|CrimeCode|count|
+---------+-----+
|4E       |91822|
|6D       |68508|
|5A       |43956|
|7A       |40308|
|6J       |27670|
|6G       |26898|
|6E       |24310|
|6C       |23269|
|4C       |22455|
|5D       |14971|
|3AF      |14760|
|4B       |14501|
|4A       |13240|
|3B       |10735|
|4D       |7222 |
|5B       |6481 |
|9S       |5452 |
|6F       |5058 |
|5C       |4917 |
|6B       |4106 |
|3CF      |3776 |
|2A       |3172 |
|3AJF     |3020 |
|3AK      |2982 |
|3K       |2833 |
|7C       |2726 |
|1A       |2631 |
|3AO      |2486 |
|5F       |2192 |
|3JF      |1819 |
|3D       |1432 |
|5E       |1419 |
|6L       |1206 |
|8H       |1053 |
|3P       |873  |
|3GF      |800  |
|6A       |782  |
|3BJ      |754  |
|3CK      |591  |
|3CO      |516  |
|3NF      |445  |
|3JK      |423  |
|8AO      |413  |
|3JO      |413  |
|2B       |355  |
|7B       |291  |
|8J       |279  |
|3H       |261  |
|1K       |230  |
|3AJO     |225  |
|6H       |222  |
|3J       |216  |
|1O       

3. Which neighborhood had most crimes?

In [17]:
(df2_with_schema
.select("Neighborhood")
.where(col("Neighborhood").isNotNull())
.groupBy("Neighborhood")
.count()
.orderBy("count", ascending=False)
.show(n=1, truncate=False))

+------------+-----+
|Neighborhood|count|
+------------+-----+
|DOWNTOWN    |17818|
+------------+-----+
only showing top 1 row



4. Which month of the year had most crimes?

In [18]:
df_month = df2_with_schema.select('CrimeDateTime', substring('CrimeDateTime', 6,2).alias('month'))
(df_month
 .select('month')
 .where(col("month").isNotNull())
 .groupBy("month")
 .count()
 .orderBy("count", ascending=False)
 .show(n=1, truncate=False))

+-----+-----+
|month|count|
+-----+-----+
|08   |46322|
+-----+-----+
only showing top 1 row



5. What weapons were used? 

In [19]:
(df2_with_schema
.select("Weapon")
.where(col("Weapon") != "NA")  #.isNotNull())
.distinct()
.show(n=10, truncate=False))

+-------+
|Weapon |
+-------+
|HANDS  |
|KNIFE  |
|OTHER  |
|FIRE   |
|FIREARM|
+-------+



6. Which weapon was used the most? 

In [20]:
(df2_with_schema
.select("Weapon")
.where(col("Weapon") != "NA")
.groupBy("Weapon")
.count()
.orderBy("count", ascending=False)
.show(n=1, truncate=False))

+-------+-----+
|Weapon |count|
+-------+-----+
|FIREARM|46217|
+-------+-----+
only showing top 1 row

